In [22]:
import pandas as pd

file_1_df = pd.read_csv('resources/file_1.csv')
file_2_df = pd.read_csv('resources/file_2.csv')
file_2_clean_df = file_2_df.dropna(subset=["Latitude", "Longitude"]).copy()

Join 2 DataFrames


In [24]:
file_1_subset = file_1_df[
    ["Borehole ID",
     "Elevation (borehole_info: D)",
     "Depth to Water (hydrologic: B)"]
]
file_1_subset = file_1_subset.rename(columns={"Elevation (borehole_info: D)": "Elevation"})


# Perform the merge
merged_df = file_2_clean_df.merge(file_1_subset, on="Borehole ID", how="left")


In [25]:
merged_df.to_csv('resources/borehole_logs.csv', index=False)

In [76]:
print(merged_df['Lithology'].unique())

['Fill' 'Sandy silt' 'Silt' 'Topsoil / vegetation' 'Silty sand' 'Clay'
 'Silty clay' 'Clayey silt' 'Clayey sand' 'Sand' 'Undefined' 'Gravel'
 'Gravelly sand' 'Asphalt / concrete' 'Peat' 'Sandy gravel' 'Debris'
 'Sandy clay' 'Silty gravel' 'Sedimentary bedrock' 'Gravelly silt'
 'Gravelly clay' 'Undifferentiated rock' 'Clayey gravel'
 'Cobbles / boulders' 'Plutonic bedrock' 'Volcanic ash' 'Volcanic bedrock'
 'Metamorphic bedrock']


merge consecutive layers
📝 Example
Before:
Borehole ID	Sub Borehole Layer	Lithology	Top Depth	Bottom Depth
1	        4	                Silt	    3.0	        4.0
1	        5	                Silt	    4.0	        5.0

After:
Borehole ID	Sub Borehole Layer	Lithology	Top Depth	Bottom Depth
1	            4	            Silt	    3.0	        5.0

In [26]:
import pandas as pd

# Load data
df = pd.read_csv('resources/borehole_logs.csv')
df.columns = df.columns.str.strip()

# Sort for proper processing
df = df.sort_values(by=['Borehole ID', 'Sub Borehole Layer']).reset_index(drop=True)

# Result list
merged_rows = []

# Group by Borehole ID
for borehole_id, group in df.groupby('Borehole ID'):
    group = group.sort_values(by='Sub Borehole Layer').reset_index(drop=True)
    i = 0
    while i < len(group):
        current_row = group.iloc[i].copy()
        j = i + 1
        # Start checking consecutive rows
        while (
            j < len(group)
            and group.loc[j, 'Lithology'] == current_row['Lithology']
            and group.loc[j, 'Sub Borehole Layer'] == current_row['Sub Borehole Layer'] + (j - i)
        ):
            current_row['Bottom Depth'] = group.loc[j, 'Bottom Depth']
            j += 1
        # Set new Sub Borehole Layer (optional: store the range or lowest one)
        current_row['Sub Borehole Layer'] = group.loc[i, 'Sub Borehole Layer']
        merged_rows.append(current_row)
        i = j

# Final merged DataFrame
merged_df = pd.DataFrame(merged_rows)

# Save if needed
merged_df.to_csv("resources/merged_borehole_layers.csv", index=False)

print("Merged consecutive layers and saved to 'merged_borehole_layers.csv'")


/var/folders/v8/z62syxvx173303z_ts3j44n40000gn/T/ipykernel_7840/2903188651.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('resources/borehole_logs.csv')


Merged consecutive layers and saved to 'merged_borehole_layers.csv'


In [28]:
import pandas as pd

# Load and clean
df = pd.read_csv('resources/merged_borehole_layers.csv')
df.columns = df.columns.str.strip()

# Group and count layers per borehole
layer_counts = df.groupby('Borehole ID').size().reset_index(name='Layer Count')

# Get top 5 boreholes with the most layers
top_boreholes = layer_counts.sort_values(by='Layer Count', ascending=False).head()
max_layers = layer_counts['Layer Count'].max()
print(f"Max number of layers in a single borehole: {max_layers}")
print(top_boreholes.head())

# Filter original DataFrame
filtered_df = df[df['Borehole ID'].isin(top_boreholes['Borehole ID'])]

# Save to CSV
filtered_df.to_csv("resources/top_borehole_records.csv", index=False)
print("Saved full records of top boreholes to 'top_borehole_records.csv'")

# Distribution bins
bins = [0, 5, 10, 20, 50, 100]
labels = ['1-5', '6-10', '11-20', '21-50', '51-100']
layer_counts['Layer Range'] = pd.cut(layer_counts['Layer Count'], bins=bins, labels=labels, right=True)

# Count and percentage per group
distribution = layer_counts['Layer Range'].value_counts().sort_index().reset_index()
distribution.columns = ['Layer Range', 'Count']
distribution['Percentage'] = (distribution['Count'] / distribution['Count'].sum() * 100).round(2)

print("\nLayer Count Distribution:")
print(distribution)

Max number of layers in a single borehole: 76
       Borehole ID  Layer Count
63159        70329           76
63158        70328           67
63875        71903           65
61591        68502           65
63876        71904           63
Saved full records of top boreholes to 'top_borehole_records.csv'

Layer Count Distribution:
  Layer Range  Count  Percentage
0         1-5  79334       90.87
1        6-10   6844        7.84
2       11-20   1008        1.15
3       21-50    113        0.13
4      51-100      5        0.01


/var/folders/v8/z62syxvx173303z_ts3j44n40000gn/T/ipykernel_7840/1133239135.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('resources/merged_borehole_layers.csv')


✅ Code to Keep Only Boreholes with 1–10 Layers:

In [29]:
import pandas as pd

# Load and clean
df = pd.read_csv('resources/merged_borehole_layers.csv')
df.columns = df.columns.str.strip()

# Group and count layers per borehole
layer_counts = df.groupby('Borehole ID').size().reset_index(name='Layer Count')

# Filter to only 1-10 layers
valid_ids = layer_counts[layer_counts['Layer Count'].between(1, 10)]['Borehole ID']

# Filter the original DataFrame
filtered_df = df[df['Borehole ID'].isin(valid_ids)]

# Save or preview
filtered_df.to_csv("resources/boreholes_1_to_10_layers.csv", index=False)
print(f"Filtered boreholes with 1–10 layers: {filtered_df['Borehole ID'].nunique()} boreholes saved.")


/var/folders/v8/z62syxvx173303z_ts3j44n40000gn/T/ipykernel_7840/767122414.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('resources/merged_borehole_layers.csv')


Filtered boreholes with 1–10 layers: 86178 boreholes saved.


✅ Final Structure (Per Layer)

🔢 Inputs (Features):
- Latitude
- Longitude
- Elevation

🎯 Outputs:
- Lithology → classification
- Top Depth → regression
- Bottom Depth → regression



In [34]:
import pandas as pd

df = pd.read_csv("resources/boreholes_1_to_10_layers.csv", low_memory=False)
df.columns = df.columns.str.strip()  # Clean whitespace

print(df.columns.tolist())  # Print all column names

def reshape_borehole(group, max_layers=10):
    row = {
        'Borehole ID': group['Borehole ID'].iloc[0],
        'Latitude': group['Latitude'].iloc[0],
        'Longitude': group['Longitude'].iloc[0],
        'Elevation': group['Elevation'].iloc[0],
    }
    group = group.sort_values(by='Top Depth')
    for i in range(max_layers):
        if i < len(group):
            row[f'Lith_{i+1}'] = group['Lithology'].iloc[i]
            row[f'Top_{i+1}'] = group['Top Depth'].iloc[i]
            row[f'Bottom_{i+1}'] = group['Bottom Depth'].iloc[i]
        else:
            row[f'Lith_{i+1}'] = 'N/A'
            row[f'Top_{i+1}'] = 0.0
            row[f'Bottom_{i+1}'] = 0.0
    return pd.Series(row)

# Apply to full dataframe
reshaped_df = df.groupby('Borehole ID').apply(reshape_borehole).reset_index(drop=True)

# Save for reuse
reshaped_df.to_csv("resources/boreholes_reshaped.csv", index=False)
print("✅ Reshaped to wide format with up to 10 layers per borehole.")

['Borehole ID', 'Document ID', 'Sub Borehole Layer', 'Latitude', 'Longitude', 'Top Depth', 'Bottom Depth', 'Description', 'USCS', 'Lithology', 'Elevation', 'Depth to Water (hydrologic: B)']


/var/folders/v8/z62syxvx173303z_ts3j44n40000gn/T/ipykernel_7840/2265567470.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reshaped_df = df.groupby('Borehole ID').apply(reshape_borehole).reset_index(drop=True)


✅ Reshaped to wide format with up to 10 layers per borehole.


In [74]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Add
from tensorflow.keras.activations import gelu
from tensorflow.keras.regularizers import l2
import pickle
import os

# Load data
df = pd.read_csv("resources/boreholes_reshaped.csv")
print(f"Initial size: {df.shape}")
df.columns = df.columns.str.strip()

# Drop rows with any missing Lithology, Top or Bottom depths
# Replace None/NaN with "N/A" for Lithology, Top, and Bottom columns
target_cols = [f'Lith_{i}' for i in range(1, 11)] + \
              [f'Top_{i}' for i in range(1, 11)] + \
              [f'Bottom_{i}' for i in range(1, 11)]

df[target_cols] = df[target_cols].fillna("N/A")

# Input features
X = df[['Latitude', 'Longitude', 'Elevation']].values
print(f"Initial X size: {df.shape}")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === Targets ===
lith_label_encoders = {}
y_lith_all = []

# Fit on full dataframe to ensure consistent class count
for i in range(1, 11):
    col = f'Lith_{i}'
    le = LabelEncoder()
    df[col] = df[col].astype(str)
    le.fit(df[col])  # Fit on full column
    lith_label_encoders[col] = le
    y_encoded = le.transform(df[col])
    y_lith_all.append(to_categorical(y_encoded))


# Top & Bottom Depths
y_top_all = [df[f'Top_{i}'].values for i in range(1, 11)]
y_bottom_all = [df[f'Bottom_{i}'].values for i in range(1, 11)]

# === Train/Test split (80/20) ===
X_train, X_test, df_train, df_test = train_test_split(X_scaled, df, test_size=0.2, random_state=42)
print(f"X_train size: {X_train.shape}")

y_lith_train = [
    to_categorical(
        lith_label_encoders[f'Lith_{i+1}'].transform(df_train[f'Lith_{i+1}'].astype(str)),
        num_classes=len(lith_label_encoders[f'Lith_{i+1}'].classes_)
    )
    for i in range(10)
]

y_lith_test = [
    to_categorical(
        lith_label_encoders[f'Lith_{i+1}'].transform(df_test[f'Lith_{i+1}'].astype(str)),
        num_classes=len(lith_label_encoders[f'Lith_{i+1}'].classes_)
    )
    for i in range(10)
]

y_top_train = [df_train[f'Top_{i+1}'].values for i in range(10)]
y_top_test = [df_test[f'Top_{i+1}'].values for i in range(10)]

y_bottom_train = [df_train[f'Bottom_{i+1}'].values for i in range(10)]
y_bottom_test = [df_test[f'Bottom_{i+1}'].values for i in range(10)]

# === Build Model ===

input_layer = Input(shape=(X_train.shape[1],))
outputs = []

def lithology_block(input_layer, i, num_classes):
    x = Dense(128, activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    return Dense(num_classes, activation='softmax', name=f'lith_{i}')(x)

def top_block(input_layer, i):
    x = Dense(64, activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    return Dense(1, activation='linear', name=f'top_{i}')(x)

def bottom_block(input_layer, i):
    x = Dense(64, activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    return Dense(1, activation='linear', name=f'bottom_{i}')(x)


# Lithology classification heads
for i in range(10):
    num_classes = y_lith_train[i].shape[1]
    outputs.append(lithology_block(input_layer, i + 1, num_classes))

# Top and Bottom regression heads
for i in range(10):
    outputs.append(top_block(input_layer, i + 1))
    outputs.append(bottom_block(input_layer, i + 1))

model = Model(inputs=input_layer, outputs=outputs)

# Outputs
outputs = []

# Lithology (classification)
for i in range(10):
    num_classes = y_lith_train[i].shape[1]
    outputs.append(lithology_block(input_layer, i + 1, num_classes))

# Top and Bottom regression heads
for i in range(10):
    outputs.append(top_block(input_layer, i + 1))
    outputs.append(bottom_block(input_layer, i + 1))

# Compile
model = Model(inputs=input_layer, outputs=outputs)

losses = {f'lith_{i+1}': 'categorical_crossentropy' for i in range(10)}
losses.update({f'top_{i+1}': 'mse' for i in range(10)})
losses.update({f'bottom_{i+1}': 'mse' for i in range(10)})

metrics = {f'lith_{i+1}': 'accuracy' for i in range(10)}
metrics.update({f'top_{i+1}': 'mae' for i in range(10)})
metrics.update({f'bottom_{i+1}': 'mae' for i in range(10)})

# Emphasize underperforming lith layers
loss_weights = {
    f'lith_1': 10.0,
    f'lith_2': 5.0,
    f'lith_3': 2.0,
    f'lith_4': 1.0,
    f'lith_5': 1.0,
    f'lith_6': 0.8,
    f'lith_7': 0.7,
    f'lith_8': 0.6,
    f'lith_9': 0.5,
    f'lith_10': 0.5,
}
loss_weights.update({f'top_{i+1}': 1.0 for i in range(10)})
loss_weights.update({f'bottom_{i+1}': 1.0 for i in range(10)})

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=losses,
    loss_weights=loss_weights,
    metrics=metrics
)

# === Train ===
class EpochPrinter(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}")

model.fit(
    X_train,
    y_lith_train + y_top_train + y_bottom_train,
    validation_data=(X_test, y_lith_test + y_top_test + y_bottom_test),
    epochs=100,
    batch_size=256,
    verbose=0,  # suppress built-in output
    callbacks=[EpochPrinter()]
)

# === Save Model & Encoders ===
os.makedirs("models", exist_ok=True)
model.save("models/borehole_multi_layer_model.keras")

with open("models/lithology_label_encoders.pkl", "wb") as f:
    pickle.dump(lith_label_encoders, f)

with open("models/feature_scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("✅ Model and encoders saved.")




Initial size: (86178, 34)
Initial X size: (86178, 34)
X_train size: (68942, 3)
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
✅ Model and encoders saved.


In [75]:
from sklearn.metrics import accuracy_score
import numpy as np

# === Make Predictions ===
preds = model.predict(X_test)

print("\n🎯 Accuracy for Lithology:")
mismatches = []
total_correct = 0
total_count = 0

for i in range(10):
    y_pred = np.argmax(preds[i], axis=1)
    y_true = np.argmax(y_lith_test[i], axis=1)

    acc = accuracy_score(y_true, y_pred)
    correct = np.sum(y_pred == y_true)
    count = len(y_true)

    total_correct += correct
    total_count += count

    print(f"  Lith_{i+1}: {acc * 100:.2f}%  ({correct}/{count})")

    # Collect mismatches for Lith_1 and Lith_2 only
    if i < 2:
        encoder = lith_label_encoders[f"Lith_{i+1}"]
        y_true_labels = encoder.inverse_transform(y_true)
        y_pred_labels = encoder.inverse_transform(y_pred)
        for idx in np.where(y_true != y_pred)[0]:
            mismatches.append((idx, f"Lith_{i+1}", y_true_labels[idx], y_pred_labels[idx]))

# === Print up to 20 mismatches with real values
print("\n❌ Sample Lith_1 and Lith_2 Mismatches (up to 20):")
for idx, layer, actual, predicted in mismatches[:20]:
    print(f"  Test Index {idx} | {layer} | actual='{actual}', predicted='{predicted}'")


539/539 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🎯 Accuracy for Lithology:
  Lith_1: 35.84%  (6178/17236)
  Lith_2: 30.54%  (5264/17236)
  Lith_3: 49.70%  (8566/17236)
  Lith_4: 73.21%  (12619/17236)
  Lith_5: 85.44%  (14726/17236)
  Lith_6: 91.81%  (15825/17236)
  Lith_7: 95.42%  (16446/17236)
  Lith_8: 97.41%  (16790/17236)
  Lith_9: 98.67%  (17006/17236)
  Lith_10: 99.45%  (17142/17236)

❌ Sample Lith_1 and Lith_2 Mismatches (up to 20):
  Test Index 0 | Lith_1 | actual='Sandy silt', predicted='Silty sand'
  Test Index 1 | Lith_1 | actual='Topsoil / vegetation', predicted='Silty sand'
  Test Index 2 | Lith_1 | actual='Topsoil / vegetation', predicted='Silty sand'
  Test Index 3 | Lith_1 | actual='Clayey silt', predicted='Silty sand'
  Test Index 4 | Lith_1 | actual='Topsoil / vegetation', predicted='Silty sand'
  Test Index 5 | Lith_1 | actual='Silty gravel', predicted='Silty sand'
  Test Index 6 | Lith_1 | actual='Clayey silt', predicted='Silty sand'
  Test Index 8 | Lith_1 | actual='Sandy

Inference Script: Predict 10 Layers

In [73]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import pickle

# === Load model and saved encoders ===
model = load_model("models/borehole_multi_layer_model.keras")

with open("models/lithology_label_encoders.pkl", "rb") as f:
    lith_encoders = pickle.load(f)

with open("models/feature_scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

# === Input a new borehole ===
# Replace this with your real test input
new_borehole = pd.DataFrame([{
    'Latitude': 47.61,
    'Longitude': -122.33,
    'Elevation': 125.0
}])

# Scale input
X_new_scaled = scaler.transform(new_borehole)

# Predict
predictions = model.predict(X_new_scaled)

# Parse predictions
layer_results = []

for i in range(10):
    # Lithology: decode from softmax
    lith_pred_softmax = predictions[i][0]
    lith_class = lith_encoders[f'Lith_{i+1}'].inverse_transform([np.argmax(lith_pred_softmax)])[0]

    # Top Depth (regression output)
    top_depth = predictions[10 + i][0][0]

    # Bottom Depth (regression output)
    bottom_depth = predictions[10 + i + 10][0][0]

    layer_results.append({
        'Layer': i + 1,
        'Lithology': lith_class,
        'Top Depth': round(top_depth, 2),
        'Bottom Depth': round(bottom_depth, 2)
    })

# === Display Result ===
result_df = pd.DataFrame(layer_results)
print("\n🔮 Predicted Soil Layers:")
print(result_df.to_string(index=False))


/Users/anatolykuznetsov/PycharmProjects/GemPy model/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step

🔮 Predicted Soil Layers:
 Layer          Lithology  Top Depth  Bottom Depth
     1 Asphalt / concrete       0.01      7.250000
     2         Silty sand       5.63     13.430000
     3                N/A      11.36     20.209999
     4                N/A      13.29     19.760000
     5                N/A      16.09     23.120001
     6                N/A      15.80     19.480000
     7                N/A      12.74     14.810000
     8                N/A       8.22     10.460000
     9                N/A       4.33      5.290000
    10                N/A       2.19      2.240000
